In [ ]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
import nltk
import re
import numpy as np

def test_tokenize(text):
    nltk_tokens = []
    pattern = re.compile("[0-9]+\.")
    pattern2= re.compile(":[0-9]+")
    pattern3= re.compile("([A-Z]\.)|([\w?-?]\.)")
    pattern4 = re.compile(".*\.")

    for t in nltk.word_tokenize(text,preserve_line=True):
        if pattern2.match(t)!=None:
            nltk_tokens.append(":")
            if "." in t:
                nltk_tokens.append(t.replace(":","").replace(".",""))
            else:
                nltk_tokens.append(t.replace(":",""))
            nltk_tokens.append(".")
        elif ((pattern.match(t)!= None) or (pattern3.match(t)== None and pattern4.match(t)!=None)) and t !=".":
            nltk_tokens.append(t.replace(".",""))
            nltk_tokens.append(".")

        else:
            nltk_tokens.append(t)
    return(nltk_tokens)  

In [ ]:
def listdir_path(d):
    # Return full path of all files & directories in directory
    list_full_path = []
    for path in os.listdir(d):
        full_path = os.path.join(d, path)
        list_full_path.append(full_path)
    return list_full_path
    
# path = "C:/Masterarbeit/Data/manuel_annotated/real_text4" 
path = ".\Sequence-Labeling-for-Reference-Parsing-of-Cyrillic-Script-Scholarly-Data\Real_annotated_data\labelled_text_per_paper"
files = listdir_path(path)
print("Anzahl files: " + str(len(files)))

In [ ]:
all_ref = []
file_ids = []
for f in files:
    file = open(f,"r",encoding="utf-8")
    ref = file.read().split("\n\n")
    for i in range(len(ref)):
        file_id = f.split("\\")[-1].replace(".txt","")+f"_{i}"
        file_ids.append(file_id)
    all_ref+=ref

In [ ]:
df_data = pd.DataFrame({"Text": [], "Labels": []})

for ref in all_ref:
    tokens = []
    labels = []
#     print(ref)
    for f in ref.split("<"):
        if f == "":
            pass
        else:
            text = f.split(">")
            if len(text)>1:
                text_token = test_tokenize(text[1])
        #         new_text = " ".join(test_tokenize(text[1]))
        #         tokens+=test_tokenize(text[1])
                if "/" in text[0]:
                    for i in text_token:
                        tokens.append(i)
                        labels.append("other")
                else:
                    label = text[0]
                    for i in text_token:
                        tokens.append(i)
                        if label=="author" and i==",":
                            labels.append("other")
                        else:
                            labels.append(label)
            else:
                text_token = test_tokenize(text[0])
                for i in text_token:
                    tokens.append(i)
                    labels.append("other")
    new_text = ' '.join(tokens)
    new_labels = ' '.join(labels)

        
    df_append = pd.DataFrame({'Text' : [new_text] , 'Labels':[new_labels]})
    df_data = df_data.append(df_append , ignore_index = True) 

In [ ]:
test_text = list(df_data.Text)
# print(test_text[:3])

In [ ]:
test_labels = list(df_data.Labels)
# print(test_labels[:3])

In [ ]:
label_dict_beg = {"title":"""<title level="a">""" ,
"author":"""<author>""",
"booktitle":"""<title level="j">""",
"journal":"""<title level="j">""",
"publisher":"""<publisher>""",
"volume":"""<biblScope unit="volume">""",
"year":"""<date>""",
"address":"""<pubPlace>""",
"number":"""<biblScope unit="issue">""",
"pages":"""<biblScope unit="page">""",
"pagetotal":"""<biblScope unit="page">"""}

label_dict_end = {"title":"""</title>""" ,
"author":"""</author>""",
"booktitle":"""</title>""",
"journal":"""</title>""",
"publisher":"""</publisher>""",
"volume":"""</biblScope>""",
"year":"""</date>""",
"address":"""</pubPlace>""",
"number":"""</biblScope>""",
"pages":"""</biblScope>""",
"pagetotal":"""</biblScope>"""}



begin_file = """<?xml version="1.0" ?>
<TEI xml:space="preserve" xmlns="http://www.tei-c.org/ns/1.0" xmlns:xlink="http://www.w3.org/1999/xlink" 
 xmlns:mml="http://www.w3.org/1998/Math/MathML">
	<teiHeader>
		<fileDesc xml:id="0"/>
	</teiHeader>
	<text>
		<front/>
		<body/>
		<back>
<listBibl>"""
end_file = """		</listBibl>
	</back>
	</text>
</TEI>"""

In [ ]:
tei_ref = []
for ref_text, ref_labels in zip(test_text,test_labels):
    new_ref_text = ""
    for ref_token, label in zip(ref_text.split(" "),ref_labels.split(" ")):
        if label == "other":
            new_ref_text+=ref_token + " "
        else:
            new_ref_text+=label_dict_beg[label]+ref_token+label_dict_end[label]+ " "
    new_ref_text = new_ref_text.replace("<i/>","").replace("\x16","--").replace("< i / >","").replace("""< ""","""&gt; """).replace(""" >""",""" &lt;""") # Anpassungen bewerten..
    new_ref_text = new_ref_text.replace(" &","&amp; ").replace("& ","&amp; ")
    tei_ref.append(new_ref_text)

In [ ]:
# path_output = "C:/Masterarbeit/Data/manuel_annotated/grobid_evaluation_token_level"
path_output = ".\Sequence-Labeling-for-Reference-Parsing-of-Cyrillic-Script-Scholarly-Data\Grobid\grobid_test_data"

for r,f in zip(tei_ref,file_ids):
    output_text = begin_file + "\n<bibl>" +r+"</bibl>\n" +end_file
    myfile = open(os.path.join(path_output, f+".references.tei.xml"), "w",encoding="utf-8")
    myfile.write(output_text)
    myfile.close()
print("DONE")
print(str(len(tei_ref))+ " files created!")

files_grobid = listdir_path(path_output)

In [ ]:
# Manually look into error files and remove errors!
errors = []
for f in files_grobid:
    try:
        tree = ET.parse(f)
        root = tree.getroot()
    except:
        
        errors.append(f)
#         os.remove(f)
print("errors in:  " + str(len(errors)) + " files. Files removed!")

In [ ]:
errors